In [6]:
import sys
from pathlib import Path
sys.path.append(str(Path().resolve().parent))
from datetime import datetime as dt
import pandas as pd
from db.db_operations import execute_db_operations

from generators.full_generators_coa import (
    create_company_data,
    create_all_erp_data,
    create_mapping_between_all, 
)

company_name = "BioCirc"

In [ ]:
data = create_company_data(company_name=company_name, save_to_csv=True)

Generating data for company: BioCirc...
Context data for BioCirc generated: {'company_name': 'BioCirc', 'count_employee': 290, 'count_department': 15, 'count_customer': 50, 'count_product': 120, 'count_procurement': 100, 'count_service': 80, 'count_account': 40, 'count_vendor': 60, 'estimated_product': 297000000, 'estimated_service': 45000000, 'estimated_overhead': 192200000, 'estimated_revenue': 1145400000}
Generating dimensions for company: BioCirc.
This usually takes 2-5 mins.

=== Attempt 1 ===
✔ Roles and Names generated.


In [2]:
df_map_products = pd.read_csv("data/outputdata/mapping/map_products.csv")
df_map_expenses = pd.read_csv("data/outputdata/mapping/map_expenses.csv")
df_erp_products = pd.read_csv("data/outputdata/mapping/erp_products.csv")
df_erp_expenses = pd.read_csv("data/outputdata/mapping/erp_expenses.csv")


In [3]:
data  =  {
        "df_erp_expenses": df_erp_expenses,
        "df_map_expenses": df_map_expenses,
        "df_erp_products": df_erp_products,
        "df_map_products": df_map_products,
    }

In [4]:
erp = create_all_erp_data(data, company_name=company_name, save_to_csv=True)

✔ All erp-data generated.
✔ All ERP CSVs saved to: data/outputdata/fact


In [5]:
df_bu = pd.read_csv("data/outputdata/dimensions/bu.csv")
df_bu.rename(columns={"bu_key": "name"}, inplace=True)
df_bu.to_csv("data/outputdata/dimensions/bu.csv", index=False)

In [4]:
df_gl = pd.read_csv("data/outputdata/fact/general_ledger.csv")
df_gl

C:\Users\jsteensgaard\AppData\Local\Temp\ipykernel_22488\2927281081.py:1: DtypeWarning: Columns (0,1,6,8,9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_gl = pd.read_csv("data/outputdata/fact/general_ledger.csv")


,document_number,debit_credit,date,amount,quantity,account_id,product_id,procurement_id,service_id,bu_id,vendor_id,customer_id,version
0,78001747,NaN,2024-05-24,-11974.1,4.0,Raw Materials,NaN,Wood chips (biomass quality),NaN,BIOCIRC_BIOGAS_PRODUCTION,FiberFlow Biomass,NaN,Actual
1,78001747,NaN,2024-05-24,-11974.1,4.0,Raw Materials,NaN,Wood chips (biomass quality),NaN,BIOCIRC_BIOGAS_PRODUCTION,CelluForm Suppliers,NaN,Actual
2,78001747,NaN,2024-05-24,-11974.1,4.0,Raw Materials,NaN,Wood chips (biomass quality),NaN,BIOCIRC_BIOGAS_PRODUCTION,BioOrganic Feedstock,NaN,Actual
3,78001012,NaN,2024-12-10,-11987.8,7.0,Raw Materials,NaN,Wood chips (biomass quality),NaN,BIOCIRC_BIOGAS_PRODUCTION,FiberFlow Biomass,NaN,Actual
4,78001012,NaN,2024-12-10,-11987.8,7.0,Raw Materials,NaN,Wood chips (biomass quality),NaN,BIOCIRC_BIOGAS_PRODUCTION,CelluForm Suppliers,NaN,Actual
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2131224,BUDG78003994,Debit,2025-12-01,300.0,3.0,Retail Gross Sales,Plant Maintenance Procedure Posters,NaN,NaN,NaN,NaN,Urban Health Initiative,Budget
2131225,BUDG78003994,Debit,2025-12-01,267900.0,5.0,Trade Gross Sales,Agricultural Digestate Delivery,NaN,NaN,NaN,NaN,BIOCIRC_BIOFERTILIZER_SALES,Budget
2131226,BUDG78003994,Debit,2025-12-01,285900.0,5.0,Trade Gross Sales,Agricultural Digestate Delivery,NaN,NaN,NaN,NaN,BIOCIRC_BIOGAS_PRODUCTION,Budget
2131227,BUDG78003994,Debit,2025-12-01,292200.0,5.0,Trade Gross Sales,Agricultural Digestate Delivery,NaN,NaN,NaN,NaN,BIOCIRC_POWER_SALES,Budget


In [8]:
now = dt.now()
version_tag = (
    company_name.lower()
    + "_final_" 
    + "_date_" + now.strftime("%m%d")
    + "_time_" + now.strftime("%H%M%S")
)
execute_db_operations(version_tag)

[INFO] Inserting 15 rows into dim_department using to_sql...
[INFO] Inserting 64 rows into dim_customer using to_sql...
[INFO] Inserting 120 rows into dim_product using to_sql...
[INFO] Inserting 109 rows into dim_account using to_sql...
[INFO] Inserting 120 rows into dim_procurement using to_sql...
[INFO] Inserting 80 rows into dim_service using to_sql...
[INFO] Inserting 14 rows into dim_payline using to_sql...
[INFO] Inserting 69 rows into dim_vendor using to_sql...
[INFO] Inserting 9 rows into dim_bu using to_sql...


c:\Users\jsteensgaard\OneDrive - KPMG\Documents\Synthetic_Data_Generation\db\db_operations.py:188: DtypeWarning: Columns (0,1,6,8,9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_erp = pd.read_csv("data/outputdata/fact/general_ledger.csv")
c:\Users\jsteensgaard\OneDrive - KPMG\Documents\Synthetic_Data_Generation\db\db_operations.py:61: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace(mapping)
c:\Users\jsteensgaard\OneDrive - KPMG\Documents\Synthetic_Data_Generation\db\db_operations.py:61: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_op

[INFO] Inserting 2131229 rows into fact_general_ledger using to_sql...


In [6]:
import pandas as pd
df = pd.read_csv("data/outputdata/dimensions/account_coa.csv")

In [17]:
df_erp_all = pd.read_csv("data/outputdata/fact/general_ledger_mapped.csv", sep=",")

In [18]:
df_erp_all

,document_number,debit_credit,date,amount,quantity,account_id,product_id,procurement_id,service_id,vendor_id,customer_id
0,78002375,Credit,2022-01-03,45000.00,100,Raw Materials,-1,1011,-1,542,-1
1,78001778,Credit,2022-02-19,45000.00,100,Raw Materials,-1,1011,-1,542,-1
2,78002090,Credit,2022-03-14,43200.00,96,Raw Materials,-1,1011,-1,542,-1
3,78002585,Credit,2022-04-03,45000.00,100,Raw Materials,-1,1011,-1,542,-1
4,78001182,Credit,2022-05-07,5850.00,13,Raw Materials,-1,1011,-1,542,-1
...,...,...,...,...,...,...,...,...,...,...,...
53774,78003999,Debit,2025-08-18,-36095.23,-1,Goodwill,-1,-1,-1,-1,-1
53775,78003999,Debit,2025-08-18,-403503.28,-1,Prepaid Expenses,-1,-1,-1,-1,-1
53776,78003999,Debit,2025-08-18,-269028.71,-1,Finished Goods,-1,-1,-1,-1,-1
53777,78003999,Debit,2025-08-18,-51937.86,-1,Office Furniture & Equipment,-1,-1,-1,-1,-1
